# Clusteranalyse mit DBSCAN

Die Clusteranalyse ist eine Art des unüberwachten Lernens. Es liegt hierbei eine Datenmenge vor, die aus mehreren Merkmalen besteht. Anders als beim überwachten Lernen gibt es allerdings kein durch die Fragestellung gegebenes Zielmerkmal. 

Ziel einer Clusteranalyse ist es, Gruppen zu finden, die ähnliche Eigenschaften besitzen, sogenannte Cluster. Manchmal ist es nützlich, für jedes Cluster typische Repräsentanten angeben zu können, dies sind dann die Clusterzentren.

Ein möglicher Anwendungsbereich von Clusteranalysen ist beispielsweise die Analyse sozialer Netzwerke bezüglich ihrer Nutzer:innen. 


<hr style="border:1px solid gray"> </hr>

## Inhalt    

1. [DBSCAN](#kap1)  
    1.1 [Durchführung](#kap11)  
    1.2 [Einfluss der Hyperparameter](#kap12)  
    1.3 [Unterschiedliche Dichte der Punktwolke](#kap13)
    
    
2. [Fazit](#kap2)  

<hr style="border:1px solid gray"> </hr>

## 1. DBSCAN <a name="kap1"></a>

### 1.1 Durchführung <a name="kap31"></a>

Ein Verfahren, das komplexere Cluster finden kann, ist DBSCAN. Das steht für "Density Based Spatial Clustering of Applications with Noise" (dt.: dichte-basierte räumliche Clusteranalyse mit Rauschen). 

Wie der Name sagt, handelt es sich um ein Dichte-basiertes Verfahren. Dichte meint hier die Anzahl der Punkte, die "nah beieinander liegen". Was das bedeutet, wird mithilfe von zwei Parametern festgelegt: Der Radius `eps` gibt die maximale Entfernung der Punkte zueinander an und bestimmt zusammen mit der minimalen Anzahl der Punkte `min_samples`, die in einem Kreis mit Radius `eps` liegen müssen, die Dichte. 

Das Verfahren beginnt mit einem beliebigen Punkt und prüft, ob es mindestens `min_samples` Punkte gibt, die nicht weiter als `eps` entfernt sind. Ist dies nicht der Fall, so wird der Punkt als "Rauschpunkt" (engl. noise) deklariert und keinem Cluster zugeordnet, sondern aussortiert und mit dem nächsten Punkt erneut begonnen. 

Gibt es dagegen mindestens `min_samples` Punkte in der Umgebung, dann wird der Punkt zum Startpunkt eines Clusters, und die Punkte in seiner `eps`-Umgebung werden diesem Cluster zugeordnet. 

Diese so dem Cluster zugeordneten Punkte werden im nächsten Schritt bezüglich `eps` und `min_samples` betrachtet. Sind die Kriterien erfüllt, werden diese Punkte "Kernpunkt" genannt und die in ihrer `eps`-Umgebung liegenden Punkte dem Cluster zugeordnet.
Ist ein Punkt einem Cluster zugeordnet, hat jedoch selbst nicht mindestens `min_smaples` Punkte in seiner `eps`-Umgebung, so wird er als Randpunkt bezeichnet.

Auf diese Weise können alle sogenannten dichte-erreichbaren Punkte (also Punkte in der `eps`-Umgebung) und Kernpunkte ermittelt und dem Cluster zugeordnet werden. Können für ein Cluster keine weiteren dichte-erreichbaren Punkte mehr gefunden werden, so wird ein beliebiger weiterer Punkt betrachtet. Sofern es sich bei diesem Punkt nicht um einen Rauschpunkt handelt, ist dies der Startpunkt für das nächste Cluster. Alle diesem neuen Cluster zugehörigen Punkte werden dann wie oben beschrieben ermittelt. 

Das Verfahren endet, wenn alle Punkte des Datensatzes entweder als Kernpunkt, als Randpunkt oder als Rauschpunkt deklariert worden sind.

Zunächst wird eine Funktion definiert, um die Ergebnisse des zu betrachtenden Verfahrens anschaulich zu plotten.
Aufgrund der Struktur der Ausgabe von DBSCAN ist die Programmierung etwas aufwändig. Der Code ist in den Kommentaren erläutert. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline
from sklearn import cluster, datasets, mixture
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def plot_dbscan(dbscan, X):

    kern =  X[dbscan.core_sample_indices_] # Kernpunkte
    rausch = X[dbscan.labels_ == -1] # Rauschpunkte
    
    # Randpunkte bestimmen:
    indizes = np.array([i for  i in range(len(X))]) # ndarray an Indizes. Es sollen Indizes für Kern- und Rauschpunkte rausgelöscht werden
    rausch_indizes, = np.where(dbscan.labels_ == -1) # Indizes der Rauschpunkte in X
    kern_rausch_indizes = np.append(dbscan.core_sample_indices_, rausch_indizes) # Indizes der Rausch- und Kernpunkte in X
    indizes_rand = np.delete(indizes, kern_rausch_indizes) # Wenn man Rausch- und Kernpunkte aus den Indizes entfernt, bleiben Randpunkte übrig
    rand = X[indizes_rand] # Genau die Randpunkte auswählen
        
    #Kernpunkte plotten (mit *)
    plt.scatter(kern[:, 0], kern[:, 1], marker='*', s=30, c=dbscan.labels_[dbscan.core_sample_indices_])
    # Rauschpunkte plotten (mit roten Xen)
    plt.scatter(rausch[:, 0], rausch[:, 1], c="r", marker="X", s=30)
    # Randpunkte plotten (mit .)
    plt.scatter(rand[:, 0], rand[:, 1], c=dbscan.labels_[indizes_rand], marker=".", alpha=0.5)
    
    plt.xlabel("$x_1$", fontsize=14)
    plt.ylabel("$x_2$", fontsize=14, rotation=0, labelpad=20)
    plt.title("eps={:.2f}, min_samples={}".format(dbscan.eps, dbscan.min_samples), fontsize=14)
    plt.show()
    

In [ ]:
from sklearn.cluster import DBSCAN

Zunächst wird DBSCAN auf den gleichen Datensatz wie oben angewendet. Hier sind keine Vorgaben nötig, die Anzahl der Cluster wird vom Algorithmus bestimmt:

In [ ]:
blob_centers = np.array(
    [[1,  2],
     [4 , 5],
     [3,  1],
     [2.2,  2.8],
     ])
blob_std = np.array([0.3, 0.2, 0.4, 0.2])
X, y = datasets.make_blobs(n_samples=500, centers=blob_centers, cluster_std=blob_std, random_state=7)  
print(X[0:5,:])
print(y[0:5])

In [ ]:
dbscan_blobs = DBSCAN(eps=0.5, min_samples=5)  
dbscan_blobs.fit(X)

In [ ]:
plot_dbscan(dbscan_blobs, X)

Der Plot zeigt:
  - Es wurden drei Cluster gefunden. Jede Punktemenge mit Sternen in der gleichen Farbe ist ein Cluster. 
  - Die runden Punkte kennzeichnen alle Randpunkte. (Hinweis: Da nicht alle Cluster Randpunkte haben, werden die Randpunkt des gelben Clusters hier fälschlicherweise lila gekennzeichnet.)
  - Die roten Kreuze zeigen Rauschpunkte, auch Ausreißer genannt. 
    
An der Überschrift ist abzulesen, welche vorbelegten Parameter DBSCAN verwendet hat: 
  - `eps` ist der Radius des Kreises, der um jeden Punkt gezogen wird.
  - `min_samples` gibt an, wie viele Punkte in diesem Kreis liegen müssen, damit ein Punkt als Kernpunkt oder Randpunkt gilt. 
    
DBSCAN berechnet keine Clusterzentren!

<div class="alert alert-block alert-success">
<b>Arbeitsauftrag:</b> 

Verändern Sie `eps` und `min_samples` mit dem Ziel, die vier Cluster zu finden, die im vorhergehenden Plot mit `eps=0.5` und `min_smaples=5` zu erkennen sind, aber nicht gefunden werden. Gibt es eine Kombination, die das ermöglicht?

</div>

In [ ]:
# Platz für Arbeitsauftrag

### 1.2 Einfluss der Hyperparameter <a name="kap12"></a>

Nun soll eine andere Datenmenge betrachtet werden: 

In [ ]:
X_neu = np.load("cluster.npy")

In [ ]:
plt.scatter(X_neu[:, 0],X_neu[:, 1]);
plt.xlabel("$x_1$", fontsize=14)
plt.ylabel("$x_2$", fontsize=14, rotation=0, labelpad=20)
plt.show()

Die Wahl von `eps` und `min_samples` kann anhand des Plots getroffen werden. Es wird der Bereich mit der niedrigsten Dichte, der dennoch als Cluster angesehen wird, gewählt. Im linken Bereich sind z.B. etwa 30 Punkte in einem Radius von 0.6.

Zunächst werden diese grafisch ermittelten Werte genutzt:

In [ ]:
dbscan_X_neu = DBSCAN(eps=0.6, min_samples=30)  
dbscan_X_neu.fit(X_neu)
plot_dbscan(dbscan_X_neu, X_neu)
plt.show()

Bei einem größeren Radius wird das kleine linke Cluster dem großen zugeordnet:

In [ ]:
dbscan_X_neu = DBSCAN(eps=0.8, min_samples=30)  
dbscan_X_neu.fit(X_neu)
plot_dbscan(dbscan_X_neu, X_neu)
plt.show()

Bei kleinerem `eps` wird das große Cluster auseinandergerissen:

In [ ]:
dbscan_X_neu = DBSCAN(eps=0.4, min_samples=30)  
dbscan_X_neu.fit(X_neu)
plot_dbscan(dbscan_X_neu, X_neu)
plt.show()

Zum Vergleich wird hier noch das Ergebnis des k-Means-Algorithmus geplottet:

In [ ]:
from sklearn.cluster import KMeans
k = 3
kmeans_neu = KMeans(n_clusters=k, random_state=0).fit(X_neu)

In [ ]:
def plot_kmeans(X, mod):                                     # mod ist das trainierte Modell
    plt.scatter(X[:,0], X[:,1], c=mod.labels_.astype(float)) # mod.labels_ ist dasselbe wie y_pred
    plt.scatter(mod.cluster_centers_[:, 0], mod.cluster_centers_[:, 1], marker='x', s=2, linewidths=12, color='k')
    plt.show()

In [ ]:
plot_kmeans(X_neu,kmeans_neu)

Hier zeigt sich wieder das gleiche Phänomen wie oben: k-Means trennt nach Distanzen nicht nach Dichte.

### 1.3 Unterschiedliche Dichte der Punktwolke <a name="kap13"></a>

`eps` und `min_samples` legen die Dichte der gesuchten Cluster fest. Sind die Cluster unterschiedlich dicht, hat der DBSCAN Schwierigkeiten: Es können immer nur Teile der Cluster gefunden werden:

In [ ]:
blob_centers = np.array(
    [[1,  1],
     [1,  1.7],
     [3,  3],
     ])
blob_std = np.array([0.1, 0.1, 0.5])

In [ ]:
def plot_clusters(X):
    plt.scatter(X[:, 0], X[:, 1])
    plt.xlabel("$x_1$")
    plt.ylabel("$x_2$")
    plt.show()

In [ ]:
X_d, y_d = datasets.make_blobs(n_samples=500, centers=blob_centers, cluster_std=blob_std, random_state=7)  
print(X_d[0:5,:])
print(y_d[0:5])
plot_clusters(X_d)

Nun wird `eps` variiert:

In [ ]:
for i in (0.1, 0.2, 0.3):
    dbscan = DBSCAN(eps=i, min_samples=5)
    dbscan.fit(X_d)
    plot_dbscan(dbscan,X_d)

Für eps = 0.1 findet DBSCAN die beiden kleinen Cluster unten links, aber nicht das große weniger dichte Cluster rechts. Wird der Radius erhöht, werden die beiden kleinen Cluster zu einem zusammengefasst. Das große rechte Cluster wird nun erkannt. 

## 2. Fazit <a name="kap2"></a>

Es wurde die Clusterananalyse mit DBSCAN auf einem selbsterzeugten Beispieldatensatz angewendet. Hierbei wurde...

- gezeigt, wie das dichte-basierte Verfahren sich inhaltlich vom distanz-basierten Clustern mit k_Means unterscheidet,

- eine visuelle Darstellung des Ergebnisses von DBSCAN thematisiert,

- der Einfluss der Hyperparameter `eps`und `min_samples`gezeigt. 